In [308]:
import pdfplumber
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np
import re
from scipy import spatial

In [175]:
import dtale
#handy pandas visualizer, when on local Jupyter Server just run dtale.show(df)
#from Docker container visual is available at http://localhost:40000/

In [286]:
path = '../datasets/resume_samples.txt'
df = pd.DataFrame()
counter = 0
id_list = []
title_list = []
skill_list = []
#exp_list = []
with open(path, 'r',encoding="utf8", errors='backslashreplace') as f:
    for line in f.readlines():
        
        splitted = line.split(':::')
        title = splitted[1].split(';')[0]
        if 'data scientist' in title.lower():
            splitted = splitted[2].lower()
            splitted = splitted.replace('\\xa0',';;;')
            splitted = splitted.replace('*',';;;')
            splitted = splitted.replace('\\x95',';;;')
            splitted = splitted.replace('education',';;;')
            #splitted = splitted.replace(',',';;;')
            splitted = splitted.split(';;;')
            
            for skill in splitted[1:]:
                if (len(skill)>1) & ('responsibilities:' not in skill.lower()):
                    id_list.append(counter)
                    
                    title_list.append(title)
                    skill_list.append(skill)
            counter+=1
df['id']=id_list
df['title']=title_list
df['skill']=skill_list

In [289]:
df

,id,title,skill
0,0,Data Scientist/Analytics Consultant,"design sql, r & python scripts to directly qu..."
1,0,Data Scientist/Analytics Consultant,aid in the development of predictive modeling...
2,0,Data Scientist/Analytics Consultant,test model fit using cross-validation and boo...
3,0,Data Scientist/Analytics Consultant,develop in an scrum environment with team mem...
4,0,Data Scientist/Analytics Consultant,"model data using pandas/r dataframes, powerpi..."
...,...,...,...
5649,66,Data Scientist,"development methodologies agile/scrum, uml, d..."
5650,66,Data Scientist,"build tools jenkins, toad, sql loader, maven,..."
5651,66,Data Scientist,reporting tools ms office (word/excel/power p...
5652,66,Data Scientist,"databases microsoft sql server 2008,2010/2012..."


In [290]:
def preprocess_for_bert(string):
    string = string.strip("\n.’:")
    string = string.strip("’")
    string = string.strip("\\n")
    string = string.replace("/"," ")
    return(string)

In [291]:
k21 = pickle.load(open('../Experiments/k21.sav', 'rb'))
objectives = []
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

2021-09-19 19:53:39,739 - INFO     - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
2021-09-19 19:53:40,039 - INFO     - Use pytorch device: cpu


In [292]:
skill_list = []
for s in df['skill'].to_list():
    skill_list.append(preprocess_for_bert(s))
embeddings = model.encode(skill_list)
clusters =  k21.predict(np.array(embeddings.tolist()))
df['cluster']=clusters

Batches:   0%|          | 0/177 [00:00<?, ?it/s]

In [320]:
for index, row in df.iterrows():
    #print(clusters[index])
    #print(embeddings[index])
    #k21.cluster_centers_[1]
    result = 1 - spatial.distance.cosine(clusters[index], embeddings[index])
    print(result)
    #print(df[skill].loc[index])

0.012538200721812554
-0.0020923825148086994
0.002754272084437126
-0.012042234284311615
0.006781751309136275
0.004793353166133718
-0.01270292835145903
-0.015644611404735276
0.010625725556774657
-0.01587119931738945
0.00032415651074857443
-0.004412240644281695
-0.0038075370334083036
-0.001122472925933149
-0.0010201533687961284
-0.0017592510873727374
0.007808212956445737
-0.009632683905703043
0.00710005598841168
nan
0.0024953181074723885
0.0024847988895894124
0.00426419360034469
-0.003929082722232025
0.010490804234957296
0.0009681376958788901
0.010710447356061747
0.006389436916389268
0.0026408363233668686
0.007748983101412366
-0.003289816958568892
0.0005896155948824644
0.00364814611818709
0.009948798691144622
0.003930541757816375
-0.008539901190240728
-0.0013018552801191685
-0.0036348745846448427
0.005108747960884341
0.012334459195632252
0.008895151681578617
0.011174079810533644
0.004747839412172272
0.010340568672370432
0.0017829056654180553
-0.0009975106681578971
0.01287586846694877
0.00

0.009496830399876144
0.024039811971982128
0.0007131802965567591
0.0026763618485470264
-0.010226078738297062
0.008899724498652506
0.00913184399923106
0.007742766891314479
0.0006064864789667723
-0.0009108668789066243
0.0035253888859603233
0.0029593969431318534
0.009153425088960998
-0.01049560928733917
0.00359890565027976
0.01019241965294504
-0.0016589719143527137
0.01227684010326735
-0.0027001034852245365
-0.0048872991850201775
-0.01274185987852361
-0.00955338993388799
-0.002888575493125156
-0.006734761325782657
0.004830285732001838
0.0020816332309704366
0.012116166991349986
-0.010226689554950763
0.006481504837135277
-0.0030067180553603468
0.011328322398522106
0.00829431804638081
0.0021393414282244727
0.0045204257029097805
nan
-0.0038276744782976824
-0.0006507756012423815
-0.010767944281756314
-0.013253749243847368
0.0014876901954968824
0.0013116135560852227
-0.008955213547954921
-0.006371986293633114
0.012249962013437643
0.011144055702892897
0.005585898755177565
-0.019338312168668104
-0

0.0069239871971730205
0.0030021495477202675
0.003576265025843073
0.0033738493004659142
0.004269190285908553
0.002747812836132235
-0.003677722129494354
0.0049670066327294204
0.004216486311007905
0.004671019038419599
-0.0014733505573869543
0.010028719399713726
0.008488312674415965
0.010341151212841293
0.0032167516135638285
0.013393265785669461
0.002576579519412814
0.008929745068500416
0.0053364858589295405
0.006288055504716783
0.008129142572037784
-0.004319036246128061
0.010246319916727642
0.010901378888918511
0.0007223848814392753
0.008750005704173391
0.006696173434435626
0.0033144424460540556
-0.00969233692125604
0.003229979407964434
-0.0006029933001541199
0.00986403518323098
-0.0035650405780356387
0.010356682228815917
-0.01621821018511116
0.008670870005678566
0.0020110046965032824
0.007420046019580462
-0.007682481727720569
0.011056735213692104
0.01804661041594202
0.004167558100756863
0.0006210444949769078
0.010424756510481914
0.001602884229954249
0.0030392063852923146
-0.0098867485533

0.0035310884088642913
-0.00736341615034708
0.008130275932315412
0.008928165354031514
0.006491546165256823
0.0015788498518991556
-0.005197661996539704
0.0065690634156351235
-0.003983109675999463
0.010550809792092442
0.004354116342214498
-0.00873749292413617
-0.010983802628747208
-0.0025081918495688527
-0.0037067523736304953
-0.016036896251782062
-0.00019767909000978356
-0.009881901875054888
0.0014149509638258229
0.0045766407191950575
-0.002470127528630739
-0.0016815037476958228
0.0031861663499522797
0.008043709973487911
0.010397851858959029
0.005848282302714725
0.005069032719074662
0.0017665686229527688
-0.01127862397408319
0.005216707663475484
-0.00602742452816174
0.007712108766863013
0.0025219030028162948
0.005867849221826815
0.0062131414361269455
-0.0011272802479702637
0.0046574037007151015
-0.0006274033166211446
0.0007034762993916033
0.01776935104729227
-0.005681026857280003
-0.004729276100674529
0.0022088746218204003
-0.0037250770723171733
0.010103600013463665
0.003270206358796157


-0.006977753208453619
0.004145136735257515
-0.006457391750172237
0.0022396913282431985
-0.0015499344706983997
0.0069810744067942965
0.005406290161711791
-0.007797659187783967
0.0009078007939843369
0.00046759884312708166
0.0013278588255641877
0.005915481809389611
0.0060382425815938
-0.0014275780873262978
-0.006785336908986084
0.0038677730906314656
0.002745365884786244
-0.010292035860992366
-0.0037416566845802635
0.0008711669678483913
0.001662313819443706
-0.004322285469605003
-0.004295268991133749
0.010109085262500983
nan
-0.002604172775810154
-0.0032338737421202524
-0.001732855287896795
0.0013185536882054372
-0.013544454350445934
4.106974748752101e-05
0.0021865386818602506
-0.0013130156348883926
-0.0002680507832462098
-0.005534098662009823
-0.00024185556302769307
0.0028970733152954287
0.011720095633833916
0.002832551280305795
0.00755336579039434
0.0021053745338303065
0.00023693897970489264
0.0065252221636217245
-0.015616185058142662
0.0018861935161585963
0.0019349123301055648
0.0017427

-0.013334576384411934
0.013700150520766119
-0.007677107251906179
0.0033668812379824553
0.0009343984707660402
0.006623319803913619
0.009013581048638808
0.0031156910911509517
-9.926909506896564e-05
0.0039918507253707824
0.002130176783548099
0.006446080736163928
0.003094213292342607
-0.008262207172949898
0.00871313585429967
0.012876048553724262
0.006379140272762429
-0.012911031995450406
-0.00736341615034708
-0.0045051137658871365
-0.0029817830672058676
0.0020182165309753364
0.017260862448835068
0.017116293778217884
-0.01372843716138794
0.002073693757157935
-0.007557929036882349
0.003570795285597228
-8.657323119232174e-05
-0.007134742930658522
-0.006930507269021335
-0.0010731804622028385
-0.006112119990116849
-0.004570359804718871
0.009963764632081928
-0.013433409701253396
-0.005265764555286712
0.008477739134186368
0.013079023324428007
0.001076541542075593
-0.000183089872316744
0.013819228455585919
-0.005074863494731963
0.00745295095055043
-0.0026331306862801274
-0.0036857639612726256
-0.0

0.007287711847131173
0.0021393414282244727
0.0045204257029097805
0.02492154651250922
-0.01366562832102347
0.006688288444575807
0.0043098497011888615
-0.0009418142362049853
0.011144055702892897
-0.01921208941034802
-0.004689533825968928
-6.170706386487979e-05
-0.015021325806077845
-0.011224019422641396
-0.013975922133815821
0.01954942161102835
0.0031401728061648537
-0.0006507756012423815
-0.010767944281756314
-0.013253749243847368
0.0014876901954968824
-0.006371986293633114
0.012249962013437643
0.011144055702892897
0.005585898755177565
-0.01921208941034802
-0.018512302903686884
-0.002380342544316516
-6.170706386487979e-05
-0.011227462983676828
-0.015021325806077845
-0.011224019422641396
-0.013975922133815821
-0.0006801864487089304
-0.00853326539601662
-0.008763447408845604
0.01522838042194452
0.00017036324007779768
-0.0244848657374237
0.002292857040968488
0.0005419566504905049
-0.0024175807660360427
-0.00736341615034708
-0.0046051328203731146
-0.004112804530351344
-0.006717804674309047


-0.013253749243847368
0.0014876901954968824
-0.006371986293633114
0.012249962013437643
0.011144055702892897
0.005585898755177565
-0.01921208941034802
-0.01956009402918646
-0.002380342544316516
-6.170706386487979e-05
-0.009262090651179777
-0.017260876365640776
-0.009269536841207904
-0.013975922133815821
-0.0006801864487089304
-0.006561769017729757
-0.008763447408845604
-0.0012756106034257364
0.0077547175472642005
-0.017913385645166535
0.003675933371480422
0.00798497066242998
0.004968495109319737
-0.0019262387447112062
-0.00015889722939799178
-0.005816347053621884
-0.0006507756012423815
-0.016850004695608378
-0.014331041653538046
-0.00014078960130659013
-0.0026858813998869024
-0.0008225414359572802
-0.010767944281756314
-0.002380342544316516
0.006254059275596413
0.00285410880226733
-0.0074894565989489514
-0.0027679612394395647
-0.004414825129321898
-0.00047089275293399346
0.0009124162620299803
0.008356873373454032
-0.0016815037476958228
0.0031861663499522797
0.015430091386329003
0.000170

0.002194462843508771
0.005179727937308365
-0.013809951913428709
-0.0007512641911580076
0.019578449157208855
0.00037468210718138995
0.0005276964979099574
-0.006548466761480132
-0.01015104337803252
0.00461379451272681
-0.006503528376942125
-0.007207807646174125
-0.007467503692701882
0.016501656338810844
0.009661588212717231
-0.0035671276754478676
0.00772802126252814
0.0022135707757630074
-0.0004312166716393673
0.002392990469207823
0.006749752272517173
-0.004487233071423402
0.009152697919009967
nan
-0.0065021275671157674
0.007808250352821178
-0.006976499534717684
-0.013782190412039208
-0.01906267879504453
-0.010206319666577368
-0.00604267862369956
0.00019503723327862765
-0.011747074703439164
-0.00986175383793908
0.005148441187366015
-0.014792551597193393
0.013654395823747945
0.0032119339497150845
0.007808250352821178
0.005701963403868526
-0.014833778877885617
0.005368583231991186
-0.0033774030791033915
-0.017625298874677675
-0.0074428525583465355
0.0003259438619940269
-0.00835140087948871

In [310]:
k20_bigrams = pickle.load(open('../Experiments/k20_bigrams', 'rb'))

In [198]:
bigrams = []
for cluster in clusters:
    bigrams.append(k20_bigrams[cluster])
df['bigrams_k20']=bigrams

In [298]:
d=dtale.show(df)

In [288]:
dtale.instances()

To gain access to an instance object simply pass the value from 'ID' to dtale.get_instance(ID)

ID Name                                    URL
 1      http://584bcee9ce77:40000/dtale/main/1
        http://584bcee9ce77:40000/dtale/main/1
 2      http://584bcee9ce77:40000/dtale/main/2
        http://584bcee9ce77:40000/dtale/main/2


In [293]:
#df['cluster'].value_counts()
for c, count in df['cluster'].value_counts().iteritems():
    print(str(c)+" "+str(count)+" "+k20_bigrams[c])
    print('-----------------------')
    
for i in range(0,20):
    if i not in df['cluster'].to_list():
         print('missing '+str(i))

15 1427 data visualization, large datasets, experience data, knowledge data, data visualisation, large data, big data, understanding data, data engineering, data pipelines, 
-----------------------
3 1061 software development, software engineering, best practices, data science, experience working, processing frameworks, frameworks spark, tools (e.g., standard software, ci cd,, 
-----------------------
11 847 develop processes, processes tools, tools monitor, model performance, performance data, data accuracy, analyze data, data analysis, ability develop, develop experimental, 
-----------------------
16 492 machine learning, learning techniques, learning algorithms, understanding machine, experience machine, deep learning, learning experience, strong knowledge, knowledge machine, machine learning,, 
-----------------------
7 383 relational databases, experience sql, knowledge sql, sql queries, experience databases, advanced sql, sql skills, experience relational, sql experience, python

In [294]:
cluster_matrix = []

for id in np.unique(df[['id']].values):
    cluster_counter = []
    filtered = df[df['id']==id]
    for cluster in range(0,20):
        c=0
        for index, row in filtered.iterrows():
            if row['cluster']==cluster:
                c+=1/len(filtered)
        cluster_counter.append(c)
    cluster_matrix.append(cluster_counter)

In [295]:
df_resume  = pd.DataFrame()
df_resume['id']=np.unique(df[['id']].values)
df_resume['matrix']=cluster_matrix
#df_resume = df_resume.append(pd.DataFrame(df_resume['matrix'].tolist()))
foo=pd.DataFrame(df_resume['matrix'].tolist())
df_resume = pd.concat([df_resume,foo], axis=1)
counted = df_resume.iloc[:,2:].astype(bool).sum(axis=0).sort_values(ascending=False)

In [296]:
scored = df_resume.iloc[:,2:].sum(axis=0).sort_values(ascending=False)

In [297]:
for cluster, score in scored.iteritems():
    print("cluster "+str(cluster)+" scored "+str(score)+" Most common bigrams of this cluster are "+k20_bigrams[cluster])
    print('---------------------')

cluster 15 scored 14.830993416249223 Most common bigrams of this cluster are data visualization, large datasets, experience data, knowledge data, data visualisation, large data, big data, understanding data, data engineering, data pipelines, 
---------------------
cluster 3 scored 13.658230137730028 Most common bigrams of this cluster are software development, software engineering, best practices, data science, experience working, processing frameworks, frameworks spark, tools (e.g., standard software, ci cd,, 
---------------------
cluster 11 scored 9.86464014150455 Most common bigrams of this cluster are develop processes, processes tools, tools monitor, model performance, performance data, data accuracy, analyze data, data analysis, ability develop, develop experimental, 
---------------------
cluster 16 scored 5.301561245213319 Most common bigrams of this cluster are machine learning, learning techniques, learning algorithms, understanding machine, experience machine, deep learning